In [1]:
import os
print(os.listdir())

['.gradio', 'akmal_fyp.ipynb', 'akmal_fyp.py', 'app.py', 'fyp_code.ipynb', 'helmet_stream.py', 'microsleep_dataset', 'trial1', 'trial2', 'trial3', 'trial4', 'trial5.zip', 'uploads', 'yolov8n.pt', 'yolov8s.pt', 'yolo_env']


In [2]:
# setup block
%pip install -q ultralytics gradio opencv-python

# import required libraries
import os
import zipfile
from ultralytics import YOLO
import gradio as gr
import cv2

# unzip dataset
dataset_zip = "trial5.zip"
dataset_dir = "trial5"
if not os.path.exists(dataset_dir):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
else:
    print("Dataset already extracted.")

Note: you may need to restart the kernel to use updated packages.


c:\Users\zacha\miniconda3\envs\yolo_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting dataset...


In [3]:
# check dataset.yaml path
yaml_path = os.path.join(dataset_dir, "dataset.yaml")
print("YAML path:", yaml_path)

YAML path: trial5\dataset.yaml


In [4]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from ultralytics import YOLO
import torch

torch.cuda.empty_cache()

model = YOLO("yolov8s.pt")  # use smallest model

results = model.train(
    data="trial5/data.yaml",
    epochs=180,
    imgsz=640,
    batch=2,
    device=0,
    half=True
)

print("✅ Training complete! Best weights at:")
print(results.save_dir)


New https://pypi.org/project/ultralytics/8.3.223 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.214  Python-3.10.18 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=trial5/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train21, nbs=64, nms=False, opset=None, 

In [36]:
import json

# ----------- GT Data Definitions -------------
gt_values = {
    "near8": [
        ("no helmet", 12.31), ("helmet false", 19.01), ("helmet not secure", 21.95), ("helmet bad clip", 14.80),
        ("helmet on head", 17.81), ("helmet front", 17.09), ("helmet poor fit", 18.49), ("helmet back", 20.97)
    ],
    "near3": [
        ("other headwear", 6.71), ("helmet false", 6.50), ("helmet not secure", 5.73), ("helmet bad clip", 5.72),
        ("helmet on head", 6.52), ("helmet poor fit", 6.33), ("helmet on head", 1.40), ("helmet back", 2.02), ("helmet on head", 0.5)
    ],
    "near7": [
        ("no helmet", 16.48), ("helmet false", 15.61), ("helmet not secure", 15.23), ("helmet bad clip", 17.51),
        ("helmet on head", 18.97), ("helmet front", 13.07), ("helmet poor fit", 12.25), ("helmet back", 15.83)
    ],
    "medium3": [  # <--- fixed from "middle3"
        ("other headwear", 6.72), ("helmet false", 6.91), ("helmet not secure", 5.33), ("helmet bad clip", 6.15),
        ("helmet on head", 6.8), ("helmet poor fit", 6.22), ("helmet on head", 1.85), ("helmet back", 1.79), ("helmet on head", 1)
    ],
    "far3": [
        ("other headwear", 7.9), ("helmet false", 6.22), ("helmet not secure", 7.13), ("helmet bad clip", 6.52),
        ("helmet on head", 7.59), ("helmet poorfit", 7.49), ("helmet on head", 1.59), ("helmet back", 1.52), ("helmet on head", 1)
    ],
    "near4": [
        ("no helmet", 7.37), ("helmet false", 8.10), ("helmet not secure", 7.76), ("helmet on head", 9.13),
        ("helmet poorfit", 7.64), ("helmet bad clip", 8.75)
    ],
    "near5": [
        ("no helmet", 15.04), ("helmet false", 14.15), ("helmet not secure", 12.90), ("helmet bad clip", 13.34),
        ("helmet on head", 13.61), ("helmet front", 12.86), ("helmet poorfit", 17.11), ("helmet back", 3.42),
        ("helmet on head", 1.8), ("helmet back", 1.5), ("helmet on head", 0.4)
    ],
    "medium4": [
        ("helmet false", 9.4), ("helmet not secure", 8.84), ("helmet bad clip", 9.80),
        ("helmet on head", 9.35), ("helmet poor fit", 8.99)
    ],
    "far4": [
        ("helmet poorfit", 8.64), ("helmet on head", 9.60), ("helmet bad clip", 9.37),
        ("helmet not secure", 9.48), ("helmet false", 9.61), ("helmet back", 9.4)
    ],
    "near1": [
        ("null", 6.01), ("no helmet", 13.54), ("helmet false", 10.69), ("no helmet", 5.17),
        ("helmet not secure", 13.79), ("helmet bad clip", 13.67), ("helmet on head", 9.69), ("helmet front", 2.65),
        ("helmet poor fit", 8.51), ("helmet back", 5.95), ("helmet front", 1.62)
    ],
    "medium1": [
        ("null", 4.23), ("no helmet", 9.69), ("helmet false", 11.5), ("helmet not secure", 12.91),
        ("helmet bad clip", 19.38), ("helmet front", 4.5), ("helmet on head", 11.85), ("helmet front", 1.58), ("helmet poor fit", 11.07), ("helmet on head", 1.85), ("helmet back", 5), ("null", 4.63)
    ],
    "far1": [
        ("null", 4.32), ("no helmet", 12.75), ("helmet false", 8.67), ("no helmet", 3.75), ("helmet not secure", 13.15), ("helmet front", 11.15), ("helmet bad clip", 11.96), ("helmet front", 10.53), ("helmet on head", 11.34), ("helmet poor fit", 10.19), ("null", 3.27)
    ],
    "near2": [
        ("other headwear", 11.17), ("helmet false", 14), ("helmet not secure", 10.98), ("helmet bad clip", 15.21),
        ("helmet on head", 14.6), ("helmet front", 8.94), ("helmet poor fit", 13.32), ("helmet back", 7.97), ("null", 2.47)
    ],
    "medium2": [
        ("other headwear", 11.76), ("helmet false", 10.69), ("helmet not secure", 11.31), ("helmet bad clip", 11.63),
        ("helmet on head", 11.01), ("helmet front", 6.77), ("helmet poor fit", 11.74), ("helmet back", 10.97), ("null", 3.58)
    ],
    "far2": [
        ("other headwear", 12.37), ("helmet false", 15.35), ("helmet not secure", 12.47), ("helmet bad clip", 13.03),
        ("helmet on head", 18.99), ("helmet front", 11.9), ("helmet poor fit", 13.29), ("helmet back", 10.75), ("null", 5.28)
    ],
    "near5order": [
        ("helmet back", 3.42), ("helmet on head", 1.8), ("helmet back", 1.5), ("helmet on head", 0.4),
        ("helmet poor fit", 17.11), ("helmet front", 12.86), ("helmet on head", 13.61), ("helmet bad clip", 13.34),
        ("helmet not secure", 12.90), ("helmet false", 14.15), ("no helmet", 15.04)
    ],
    "near6": [  # Added explicitly based on your latest near6.json review
        ("helmet false", 18.0),
        ("helmet not secure", 14.0),
        ("helmet bad clip", 18.0),
        ("helmet on head", 20.0),
        ("helmet front", 14.0),
        ("helmet poor fit", 14.0),
        ("helmet back", 4.0),
        ("other headwear", 11.0)
    ]
}

helmet_missing_aliases = {"no helmet", "other headwear", "bare", "cap", "hood", "null"}
fps = 10.0

def to_helmet_missing(klass):
    l = klass.strip().lower()
    if l in helmet_missing_aliases:
        return "helmet missing"
    if "poorfit" in l: return "helmet poor fit"
    if "bad clip" in l or "badclip" in l: return "helmet bad clip"
    return l

for fname, class_blocks in gt_values.items():
    gt_tracks = {}
    cur_time = 0.0
    for k, sec in class_blocks:
        label = to_helmet_missing(k)
        if sec == 0.0: continue   # Ignore 0s intervals like in the original near6.json
        start = cur_time
        end = start + float(sec)
        gt_tracks.setdefault("001", []).append({"class": label, "t0": start, "t1": end})
        cur_time = end
    out = {"fps": fps, "tracks": gt_tracks}
    with open(f"{fname}.json", "w") as f:
        json.dump(out, f, indent=2)
print("✅ All GT JSONs saved.")


✅ All GT JSONs saved.


In [ ]:
#cd fyp
#conda activate yolo_env
#python helmet_stream.py

In [ ]:
import cv2, gradio as gr, torch, datetime
from ultralytics import YOLO

# Load model
model = YOLO(r"C:\Users\zacha\runs\detect\train9\weights\best.pt")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Global flag for controlling webcam loop
running = False

def start_camera():
    global running
    running = True
    cap = cv2.VideoCapture(0)
    log = []
    while running:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame, verbose=False)[0]
        annotated = results.plot()

        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        detections = [results.names[int(c)] for c in results.boxes.cls]
        log.append(f"[{timestamp}] Detected: {', '.join(detections) if detections else 'None'}")

        # yield keeps streaming images + log text
        yield annotated[:, :, ::-1], "\n".join(log[-10:])
    cap.release()

def stop_camera():
    global running
    running = False
    return "🛑 Camera stopped."

with gr.Blocks() as demo:
    gr.Markdown("## 🪖 Helmet Detection (Continuous Streaming)")

    with gr.Row():
        video_out = gr.Image(label="Live Detection Feed")
        log_box = gr.Textbox(label="Detection Log", lines=10, interactive=False)

    start_btn = gr.Button("▶ Start Camera")
    stop_btn = gr.Button("⏹ Stop Camera")

    start_btn.click(start_camera, outputs=[video_out, log_box])
    stop_btn.click(stop_camera, outputs=log_box)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://aa1097f3e72fbd24dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [25]:
#Run with: python app_helmet_monitor.py
#Then open: http://127.0.0.1:7860
import gradio
print(gradio.__version__)

5.49.0
